<a href="https://colab.research.google.com/github/nribot/smoking/blob/main/Smoking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Leer archivo con los datos
* Preprocesado de los datos: eleimiar columnos que no nos interesen, limpiar * valores perdidos, cambiar etiqueta de las clases a 0, 1, 2, 3, .... en caso de * que sean strings, codificar o transformar columnas que sean texto, ....
* Separar entre X e Y
* Separar en entrenamiento y test (si no nos lo dan por defecto)
* Normalizar
* Entrenar los modelos que queramos de clasificación: Predicciones, evaluación * (alguna métrica de clasificación que hemos visto o varias de ellas)
* Comparar los resultados de todos los modeos y quedarnos con el mejor.
* Añadir validación cruzada a los hiperparámetros que considere oportuno!!!

In [ ]:
# Para visualizar gráficas en notebooks
%matplotlib inline 

#Importar algunas librerías que serán necesarias trabajando con datos y realizando gráficas
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay

### Carga de datos


Datos obtenidos de https://www.kaggle.com/datasets/kukuroo3/body-signal-of-smoking

La variable de salida es `smoking` que tiene dos valores en este dataset, según la documentación:
* 0 = no han fumado nunca
* 1 = fumaban anteriormente (pero ya no)

Había una tercera categoría, fumadores activos, que se ha eliminado ya del dataset.

In [ ]:
data = pd.read_csv('smoking.csv')
data.head()

FileNotFoundError: ignored

In [ ]:
original_dim = data.shape
print("El dataset contiene", data.shape[0], "observaciones de", data.shape[1], "variables.")

Examinamos el tipo de las columnas y si hay nulos:

In [ ]:
data.info()

### Examen y tranformación de los datos

Visualizamos las columnas categóricas, incluyendo la variable de salida.



In [ ]:
categorical = ['gender', 'oral', 'tartar', 'smoking']

for i in categorical:
  idx = categorical.index(i)
  ax1 = plt.subplot(2,2, idx+1)
  ax1.pie(data[i].value_counts(), 
          labels=data[i].unique(),
          autopct = '%1.1f%%'
          )
  ax1.set_title(i)
  

Vemos que la categoría `oral` solo tiene un valor, la eliminamos junto con la variable `ID`.

In [ ]:
data = data.drop(labels=['ID', 'oral'], axis=1)
data.shape

Además, vemos que el dataset tiene mucha más presencia de personas no fumadoras y de mujeres, vamos a examinar si hay alguna relación.

In [ ]:
CrosstabResult=pd.crosstab(index=data['gender'],columns=data['smoking'])
print(CrosstabResult)

Parece que sí hay una clara distribución por sexo: la gran mayoría de fumadores previos son hombres.

Las columnas 'gender' y 'tartar' solo contienen dos categorías; las cambiamos a variables numéricas.

* Gender: 0 = male, 1 = female
* Tartar  0 = N, 1 = Y

In [ ]:
data['gender'].replace('M', 0, inplace=True)
data['gender'].replace('F', 1, inplace=True)
data.replace('N', 0, inplace=True)
data.replace('Y', 1, inplace=True)
#data.loc[:,['gender', 'tartar']].head()

In [ ]:
data.describe()

Las columnas `eyesight(left)` y `eyesgiht(right)` tienen valroes máximos de 9.9 pero la mayoría están entre cero y dos. Creemos que esto indica valores faltantes.

Lo mismo podría pasar con la columna  `Gtp` y el valor 999, pero en ese caso sí que hay valores de más de novecientos, así que la vamos a dejar.

In [ ]:
col = [ 'eyesight(left)', 'eyesight(right)']

for i in col:
  idx = col.index(i)
  ax = plt.subplot(1,2, idx+1)
  ax.hist(data[i])
  ax.set_title(i)
plt.rcParams["figure.figsize"] = (3,2) # hacemos las gráficas más pequeñas
plt.show()

Como tenemos muchas observaciones y hay muy pocos valores faltantes, eliminamos las filas con valores faltantes.

In [ ]:
data = data.drop(data[data['eyesight(left)']==9.9].index)
data = data.drop(data[data['eyesight(right)']==9.9].index)
data.shape

In [ ]:
col = [ 'eyesight(left)', 'eyesight(right)']

for i in col:
  idx = col.index(i)
  ax = plt.subplot(1,2, idx+1)
  ax.hist(data[i])
  ax.set_title(i)
plt.rcParams["figure.figsize"] = (6,2) # hacemos las gráficas más pequeñas
plt.show()

plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"] # reset valores

In [ ]:
print("Nos hemos quedado con el", round(data.shape[0] *100 / original_dim[0],2), "% de observaciones")

### Balanceo del dataset

En la exploración hemos visto que teníamos mucha más cantidad de observaciones con el valor 0 que con el valor 1 en la variable predictora.

Deberíamos coger la misma cantidad de datos de las dos clases (smoking=0 vs smoking=1) para no desviar el modelo.

Vamos a realizar un undersampling.

In [ ]:
# undersampling
smoking1 = data[data['smoking']==1]
smoking0 = data[data['smoking']==0]
smoking0 = smoking0.sample(n=len(smoking1), random_state=101)
data = pd.concat([smoking0, smoking1],axis=0)
print("Nos hemos quedado con el", round(data.shape[0] *100 / original_dim[0],2), "% de observaciones")

### Separación variables explicatorias y predictiva

In [ ]:
X = data.iloc[:, :-1]
Y = data.iloc[:, -1]

print(X.shape)
print(Y.shape)

### División train/test/validation

Como tenemos un gran número de observaciones, vamos a hacer la validación de `k` con un set de validación en lugar de hacer validación cruzada.

In [ ]:
X0_train, X0_test, Y0_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X1_train, X1_val, Y_train, Y_val = train_test_split(X0_train, Y0_train, test_size=0.2, random_state=0)

transformer = StandardScaler().fit(X1_train)
X_train = transformer.transform(X1_train)
X_val = transformer.transform(X1_val)
X_test = transformer.transform(X0_test)
print("Observaciones de train:", X_train.shape[0]) 
print("Observaciones de validación:",X_val.shape[0])
print("Observaciones de test:",X_test.shape[0])

### Normalización

In [ ]:
transformer = StandardScaler().fit(X_train)
X_train = transformer.transform(X_train)
X_val = transformer.transform(X_val)
X_test = transformer.transform(X_test)

### Modelo con KNN

#### Entreno del modelo con varias k en el conjunto de validación

In [ ]:
k_list = [1, 10, 50, 100, 200, 1000, 10000]

# Dibujamos los errores de entrenamiento y de test en función del parámetro k
n_tr = X_train.shape[0]
pe_tr = []
pe_val = []

for k in k_list:
    # Errores de entrenamiento
    KNN_k = KNeighborsClassifier(n_neighbors=k)
    KNN_k.fit(X_train, Y_train)
    Z_tr = KNN_k.predict(X_train)
    E_tr = Z_tr.flatten()!=Y_train

    # Errores de val
    Z_tst = KNN_k.predict(X_val)
    E_val = Z_tst.flatten()!=Y_val

    # Tasas de error
    pe_tr.append(np.mean(E_tr))
    pe_val.append(np.mean(E_val))

k_opt = k_list[np.argmin(pe_val)]
print('El valor óptimo de k es ' + str(k_opt))

In [ ]:
# graficamos
plt.plot(np.log10(k_list), pe_tr,'b--o',label='Training error')
plt.plot(np.log10(k_list), pe_val,'g--o',label='Validation error')
plt.stem([np.log10(k_opt), np.log10(k_opt)], [0, min(pe_val)],'r-o',label='Optimal k')
plt.xlabel('$log(k)$')
plt.ylabel('Tasa de error')
plt.legend(loc='best')

#### Evaluación del modelo con la mejor k

In [ ]:
clf = KNeighborsClassifier(k_opt)
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)
pe_tst = np.mean(Y_test != Y_test_pred)
print('Tasa de error', round(pe_tst,2), "con k = ", k_opt)

In [ ]:
ConfusionMatrixDisplay.from_estimator(clf, X_test, Y_test)